In [25]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Load phenotype data
print("Loading phenotype data...")
phenotypes = pd.read_csv('/content/drive/MyDrive/gwas_phenotypes.txt', delimiter='\t', header=None, names=['ID', 'Phenotype'])
phenotypes = phenotypes.set_index('ID')

# Print phenotype individual IDs for inspection
print("\nPhenotype individual IDs (first 10):")
print(phenotypes.index[:10])

# Parse VCF file and extract genotype data
print("\nParsing VCF data...")
vcf_data = []
individual_ids = []  # Store individual IDs from the VCF header

try:
    with open('/content/drive/MyDrive/gwas_population.vcf') as f:
        for line in f:
            if line.startswith('#CHROM'):
                # Extract all individual IDs from the VCF header (columns 9 and onward)
                individual_ids = line.strip().split('\t')[9:]
                print(f"\nExtracted VCF individual IDs (first 10): {individual_ids[:10]}")
            elif not line.startswith('#'):
                fields = line.strip().split('\t')
                # Convert genotype data (e.g., '0|1') to sum of alleles (0, 1, or 2)
                genotypes = [sum(map(int, g.split('|'))) for g in fields[9:]]
                vcf_data.append(genotypes)
except Exception as e:
    print(f"Error while parsing VCF file: {e}")

# Create a DataFrame for VCF data (rows: SNPs, columns: Individuals)
vcf_df = pd.DataFrame(vcf_data, columns=individual_ids).T  # Transpose for correct alignment

# Ensure individuals align between VCF and phenotype data
matching_ids = vcf_df.index.intersection(phenotypes.index)
vcf_df = vcf_df.loc[matching_ids]
phenotypes = phenotypes.loc[matching_ids]

print(f"\nNumber of matching individuals: {len(matching_ids)}")

# (a) Calculate p-values for each SNP
print("Calculating p-values...")
p_values = []

for snp in vcf_df.columns:
    contingency_table = pd.crosstab(vcf_df[snp], phenotypes['Phenotype'])
    if contingency_table.shape[0] < 2:  # Check for variation
        p_values.append(1.0)  # Assign non-significant p-value
        continue
    _, p_value, _, _ = chi2_contingency(contingency_table)
    p_values.append(p_value)

# (b) Count the number of SNPs with uncorrected p-value < 0.05
num_significant = sum(np.array(p_values) < 0.05)
expected_by_chance = 0.05 * len(p_values)

print(f"\nNumber of SNPs with uncorrected p-value < 0.05: {num_significant}")
print(f"Expected number by chance: {expected_by_chance}")

# (c) Apply Bonferroni correction and calculate odds ratios
bonferroni_threshold = 0.05 / len(p_values)
significant_snps = [(i, p) for i, p in enumerate(p_values) if p < bonferroni_threshold]

def calculate_odds_ratio(genotype, phenotype, target):
    """Calculate the odds ratio for the target genotype."""
    disease_prob = phenotype[genotype == target].mean()
    ref_prob = phenotype[genotype == 0].mean()
    return disease_prob / ref_prob if ref_prob > 0 else np.inf  # Avoid division by zero

# Prepare results for output
results = []
for snp_id, p_val in significant_snps:
    genotype = vcf_df[snp_id]
    odds_ratio_het = calculate_odds_ratio(genotype, phenotypes['Phenotype'], 1)
    odds_ratio_homo_alt = calculate_odds_ratio(genotype, phenotypes['Phenotype'], 2)

    corrected_p_value = p_val * len(p_values)  # Bonferroni correction
    results.append((snp_id, p_val, corrected_p_value, odds_ratio_het, odds_ratio_homo_alt))

# Print results in table format
# Print the results in the expected format
print("\nSignificant SNPs after Bonferroni correction:")
print(f"{'SNP ID':<10}{'Uncorrected p-value':<20}{'Corrected p-value':<20}{'OR (Het)':<30}{'OR (Homo Alt)'}")

for snp_id, p_val, corrected_p, or_het, or_homo_alt in results:
    print(f"SNP{snp_id:<8}{p_val:<20.2e}{corrected_p:<20.2e}{or_het:<30.2f}{or_homo_alt:<15.2f}")


# (d) Explanation of discrepancies in odds ratios and p-values
print("\nExplanation:")
print("SNPs might have similar odds ratios but different p-values due to:")
print("1. Sample size variance: SNPs with lower allele frequencies can yield higher p-values despite similar effect sizes.")
print("2. Multiple hypothesis testing: Statistical significance can vary after Bonferroni correction.")
print("3. Chi-squared test sensitivity: The test is sensitive to the distribution of counts across genotypes.")














Loading phenotype data...

Phenotype individual IDs (first 10):
Index(['IND0', 'IND1', 'IND2', 'IND3', 'IND4', 'IND5', 'IND6', 'IND7', 'IND8',
       'IND9'],
      dtype='object', name='ID')

Parsing VCF data...

Extracted VCF individual IDs (first 10): ['IND3', 'IND4', 'IND5', 'IND6', 'IND7', 'IND8', 'IND9', 'IND10', 'IND11', 'IND12']

Number of matching individuals: 997
Calculating p-values...

Number of SNPs with uncorrected p-value < 0.05: 227
Expected number by chance: 247.65

Significant SNPs after Bonferroni correction:
SNP ID    Uncorrected p-value Corrected p-value   OR (Het)                      OR (Homo Alt)
SNP1000    1.74e-16            8.61e-13            1.59                          2.52           
SNP2000    9.46e-07            4.68e-03            1.50                          2.25           
SNP3000    5.42e-10            2.68e-06            1.59                          2.03           
SNP4000    6.57e-07            3.26e-03            1.34                          